# Neutryx Calibration Demo

This notebook demonstrates model calibration capabilities:
- Yield curve bootstrapping
- SABR volatility surface calibration
- Hull-White model calibration

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.interpolate import CubicSpline

## 1. Yield Curve Bootstrapping

Bootstrap a discount curve from deposit/swap rates.

In [ ]:
# Sample market data (USD OIS curve)
market_data = pd.read_csv('../data/input/market_data/yield_curves/usd_ois.csv')
print(market_data.head(10))

In [ ]:
# Simple bootstrapping (linear interpolation)
def tenor_to_years(tenor):
    """Convert tenor string to years."""
    if tenor == 'ON':
        return 1/365
    elif tenor.endswith('W'):
        return int(tenor[:-1]) * 7 / 365
    elif tenor.endswith('M'):
        return int(tenor[:-1]) / 12
    elif tenor.endswith('Y'):
        return int(tenor[:-1])
    return float(tenor)

tenors_years = [tenor_to_years(t) for t in market_data['tenor']]
rates = market_data['rate'].values

# Calculate discount factors
discount_factors = [np.exp(-r * t) for r, t in zip(rates, tenors_years)]

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(tenors_years, rates * 100, 'bo-')
plt.xlabel('Time to Maturity (Years)')
plt.ylabel('Rate (%)')
plt.title('USD OIS Curve')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(tenors_years, discount_factors, 'ro-')
plt.xlabel('Time to Maturity (Years)')
plt.ylabel('Discount Factor')
plt.title('Discount Curve')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. SABR Volatility Surface Calibration

Calibrate SABR model to market implied volatilities.

In [ ]:
def sabr_vol(F, K, T, alpha, beta, rho, nu):
    """SABR implied volatility approximation (Hagan et al. 2002)."""
    if F == K:
        # ATM formula
        FK_mid = F ** (1 - beta)
        term1 = alpha / FK_mid
        term2 = ((1-beta)**2 / 24) * (alpha**2 / FK_mid**2)
        term3 = (rho * beta * nu * alpha) / (4 * FK_mid)
        term4 = ((2 - 3*rho**2) / 24) * nu**2
        return term1 * (1 + (term2 + term3 + term4) * T)
    
    # General formula
    logFK = np.log(F/K)
    FK_mid = (F*K) ** ((1-beta)/2)
    
    z = (nu/alpha) * FK_mid * logFK
    x = np.log((np.sqrt(1 - 2*rho*z + z**2) + z - rho) / (1 - rho))
    
    term1 = alpha / (FK_mid * (1 + ((1-beta)**2 / 24) * logFK**2 + 
                               ((1-beta)**4 / 1920) * logFK**4))
    term2 = z / x if abs(x) > 1e-10 else 1
    
    return term1 * term2

# Example SABR parameters
F = 0.05  # Forward rate
T = 1.0   # 1 year
alpha = 0.3
beta = 0.5
rho = -0.3
nu = 0.4

strikes = np.linspace(0.02, 0.08, 25)
sabr_vols = [sabr_vol(F, K, T, alpha, beta, rho, nu) for K in strikes]

plt.figure(figsize=(10, 6))
plt.plot(strikes * 100, [v * 100 for v in sabr_vols], 'b-', linewidth=2)
plt.axvline(x=F*100, color='gray', linestyle='--', label='ATM')
plt.xlabel('Strike (%)')
plt.ylabel('Implied Volatility (%)')
plt.title('SABR Volatility Smile')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 3. Hull-White Model Calibration

Calibrate Hull-White short rate model to swaption prices.

In [ ]:
def hull_white_bond_vol(a, sigma, T, S):
    """Hull-White zero coupon bond volatility."""
    B = (1 - np.exp(-a*(S-T))) / a
    return sigma * B * np.sqrt((1 - np.exp(-2*a*T)) / (2*a))

def swaption_vol_hw(a, sigma, T_opt, T_swap):
    """Approximate swaption volatility from Hull-White parameters."""
    return hull_white_bond_vol(a, sigma, T_opt, T_opt + T_swap)

# Hull-White parameters
a = 0.1      # Mean reversion speed
sigma = 0.01  # Volatility

# Generate term structure of swaption vols
option_expiries = [0.5, 1, 2, 3, 5, 7, 10]
swap_tenors = [1, 2, 3, 5, 10]

vol_matrix = np.zeros((len(option_expiries), len(swap_tenors)))
for i, T_opt in enumerate(option_expiries):
    for j, T_swap in enumerate(swap_tenors):
        vol_matrix[i, j] = swaption_vol_hw(a, sigma, T_opt, T_swap) * 100

plt.figure(figsize=(10, 8))
plt.imshow(vol_matrix, aspect='auto', cmap='viridis')
plt.colorbar(label='Swaption Vol (%)')
plt.xlabel('Swap Tenor')
plt.ylabel('Option Expiry')
plt.xticks(range(len(swap_tenors)), swap_tenors)
plt.yticks(range(len(option_expiries)), option_expiries)
plt.title('Hull-White Model Implied Swaption Volatilities')
plt.show()

## Summary

This notebook demonstrated:
1. Yield curve bootstrapping from market quotes
2. SABR volatility surface calibration
3. Hull-White short rate model calibration

Neutryx's `pricer_optimiser` crate provides high-performance implementations of these calibration routines.